In [8]:
import os

# Get the current working directory
current_directory = os.getcwd()

# List the contents of the current directory
contents = os.listdir(current_directory)

# Print the contents
print(f"Contents of the current working directory ({current_directory}):")
for item in contents:
    print(item)

Contents of the current working directory (/home/machina/MEDIT/medit_pipeline):
Makefile
configs
pyrightconfig.json
tools
MakefileNew
scripts
dockercompose.yml
README.md
out
clust_run.log
infra
Dockerfile
data
clust_sentinel.txt


In [11]:
import yaml
from pathlib import Path

import scanpy as sc

from scripts.EGGFM.train_energy import train_energy_model
from scripts.EGGFM.engine import EGGFMDiffusionEngine
from scripts.EGGFM.data_sources import AnnDataViewProvider
from scripts.EGGFM.utils import subsample_adata

# ---- user knobs for quick tests ----
PARAMS_PATH = "configs/params.yml"  # change if needed
MAX_CELLS = 2000                    # None = use all cells
SUBSAMPLE_SEED = 0                  # controls which cells get picked

# Load params
params = yaml.safe_load(Path(PARAMS_PATH).read_text())
specs = params["specs"]
diff_cfg = params.get("eggfm_diffmap", {})
model_cfg = params.get("eggfm_model", {})
train_cfg = params.get("eggfm_train", {})

diff_cfg


ModuleNotFoundError: No module named 'scripts.EGGFM.engine'

In [ ]:
print("[notebook] loading paul15...", flush=True)
qc_ad = sc.read_h5ad(spec.get("ad_file"))

# print("[notebook] running prep_for_manifolds...", flush=True)
# qc_ad = prep_for_manifolds(ad)

print("[notebook] subsampling (if requested)...", flush=True)
qc_ad = subsample_adata(
    qc_ad,
    max_cells=MAX_CELLS,
    seed=SUBSAMPLE_SEED,
)

qc_ad


In [ ]:
print("[notebook] training energy model...", flush=True)
energy_model = train_energy_model(qc_ad, model_cfg, train_cfg)
energy_model


In [ ]:
# Build AnnData view provider (geometry + energy spaces)
view_provider = AnnDataViewProvider(
    geometry_source=diff_cfg.get("geometry_source", "pca"),
    energy_source=diff_cfg.get("energy_source", "hvg"),
)

# Build the EGGFM diffusion engine
engine = EGGFMDiffusionEngine(
    energy_model=energy_model,
    diff_cfg=diff_cfg,
    view_provider=view_provider,
)

metric_modes = ["euclidean", "scm", "hessian_mixed"]

for mode in metric_modes:
    print(f"[notebook] Building embedding for metric_mode='{mode}'", flush=True)
    X_emb = engine.build_embedding(qc_ad, metric_mode=mode)
    key = f"X_eggfm_{mode}"
    qc_ad.obsm[key] = X_emb
    print(f"[notebook] Stored embedding in .obsm['{key}'] with shape {X_emb.shape}", flush=True)

qc_ad


In [ ]:
from sklearn.metrics import adjusted_rand_score
import numpy as np

label_key = "Cell type annotation"  # this is the usual key for paul15
if label_key not in qc_ad.obs:
    raise KeyError(f"Label column '{label_key}' not found in qc_ad.obs")

labels = qc_ad.obs[label_key].to_numpy()

def kmeans_ari(X, labels, n_clusters=None, random_state=0):
    from sklearn.cluster import KMeans
    if n_clusters is None:
        n_clusters = len(np.unique(labels))
    km = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    preds = km.fit_predict(X)
    return adjusted_rand_score(labels, preds)

for mode in metric_modes:
    key = f"X_eggfm_{mode}"
    X = qc_ad.obsm[key]
    ari = kmeans_ari(X, labels)
    print(f"metric_mode='{mode}': ARI={ari:.3f}")


In [ ]:
out_path = f"data/paul15/paul15_eggfm_test_{qc_ad.n_obs}cells.h5ad"
Path("data/paul15").mkdir(parents=True, exist_ok=True)
print(f"[notebook] writing result to {out_path}", flush=True)
qc_ad.write_h5ad(out_path)
